### 1.Import models

In [ ]:
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain import hub
from langchain.agents import create_react_agent,AgentExecutor
import requests
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search import TavilySearchResults
import os
os.environ["TAVILY_API_KEY"] = "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"

### Create a LLm

In [ ]:
llm=ChatGroq(model="llama3-70b-8192")

### Create react prompt

In [ ]:
# prompt = hub.pull("hwchase17/react")
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages({
    """
{chat_history}

'system': You must not answer by yourself; always use the available tools.
- If the user input is related to weather and includes a city, use the weather tool first.
- For all other queries, use the TravilySearch tool first.
- Use chat history to quickly answer follow-up questions based on the user's last messages.
- Provide responses as quickly as possible.

Use the following format:

Question: {input}  
Thought: Consider the appropriate action to take.  
Action: Choose one of [{tools} in {tool_names}].  
Action Input: Provide the necessary input for the chosen action.  
Observation: Record the result of the action.  
... (Repeat the Thought/Action/Action Input/Observation process as needed)  
Thought: I now know the final answer.  
Final Answer: Provide the final answer to the original question.

Begin!

Question: {input}  
Thought: {agent_scratchpad}
"""
})



### Create Tool

In [ ]:
def get_weather(city):
    api_key = "cdbd83db74eb405f306713534d0a920e" 
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response= requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return f"Weather in {data['name']}: {data['weather'][0]['description']}, {data['main']['temp']}°C"
        
    else:
        return "City not found or API error"


In [ ]:
tavily_api_key = os.getenv("TAVILY_API_KEY")
TS= TavilySearchResults(max_results=1,tavily_api_key=tavily_api_key )

In [ ]:
weather_tool = Tool(
    name="Weather Checker",
    func=get_weather,
    description="""Fetches real-time weather for a given city.                  
                """
)

### Use Chat History

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
tools=[weather_tool,TS]
agent=create_react_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,handle_parsing_errors=True,memory=memory)


### Use Agent

In [ ]:

while True:
    query=input('Ask your Queary:')
    
    if query=="quit":
        break
    
    else:  
        response = agent_executor.invoke({
            "input": query ,
            "chat_history":memory.chat_memory.messages
            })

        print("Human:",query)
        print("AI:",response['output'])

Human: population of that capitla
AI: Agent stopped due to iteration limit or time limit.


In [ ]:
for msg in memory.chat_memory.messages:
    role = "Human" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content}")
